In [3]:
# import our dependencies
import requests
import pandas as pd
import gmaps
from citipy import citipy
import numpy as np
from config2 import weather_api_key
from datetime import datetime

In [4]:
# create 2000 random latitudes and longitudes
lats = np.random.uniform(-90, 90, size=2000)
langs = np.random.uniform(-180, 180, size=2000)
lats_langs = zip(lats,langs)
coordinates = list(lats_langs)
coordinates[0]

(61.76485973266216, 148.26824319491084)

In [5]:
# Get the nearest city
# Create a list for holding the cities.
cities = []
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
            cities.append(city)
len(cities)

757

In [6]:
# create an empty list for weather data
city_data =[]
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
record_count = 1
set_count = 1
for i, city in enumerate(cities):
    print(i)
    # Group cities in sets of 50 for logging purposes.
    if (i % 50==0 and i >= 50):
        set_count += 1
    city_url = url + "&q=" + city.replace(" ","+")
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
     # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        city_weather = requests.get(city_url).json()
        # Parse the JSON and retrieve data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        city_data.append({"City": city.title(),
                      "Lat": city_lat,
                      "Lng": city_lng,
                      "Max Temp": city_max_temp,
                      "Humidity": city_humidity,
                      "Cloudiness": city_clouds,
                      "Wind Speed": city_wind,
                      "Country": city_country,
                      "Current Description": city_description,
                      "Date": city_date})
    except:
        print(f"City not found..Skipping")
        pass
    
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
0
Processing Record 1 of Set 1 | kholodnyy
1
Processing Record 2 of Set 1 | chuy
2
Processing Record 3 of Set 1 | pisco
3
Processing Record 4 of Set 1 | hilo
4
Processing Record 5 of Set 1 | grand gaube
5
Processing Record 6 of Set 1 | komsomolskiy
6
Processing Record 7 of Set 1 | nguiu
City not found..Skipping
7
Processing Record 8 of Set 1 | yulara
8
Processing Record 9 of Set 1 | saleaula
City not found..Skipping
9
Processing Record 10 of Set 1 | carnarvon
10
Processing Record 11 of Set 1 | nanortalik
11
Processing Record 12 of Set 1 | kodiak
12
Processing Record 13 of Set 1 | saldanha
13
Processing Record 14 of Set 1 | fez
14
Processing Record 15 of Set 1 | laguna
15
Processing Record 16 of Set 1 | jumla
16
Processing Record 17 of Set 1 | ushuaia
17
Processing Record 18 of Set 1 | yellowknife
18
Processing Record 19 of Set 1 | kaeo
19
Processing Record 20 of Set 1 | castro
20
Processing Record 21 of Set 1 | kapaa
21
Proces

In [7]:
print(len(city_data))

693


In [8]:
# Add data to dataframe
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Date
0,Kholodnyy,62.72,147.95,12.58,94,100,2.82,RU,light snow,2020-10-30 06:57:19
1,Chuy,-33.70,-53.46,52.90,82,100,5.93,UY,overcast clouds,2020-10-30 06:57:19
2,Pisco,-13.70,-76.22,62.01,87,75,13.87,PE,broken clouds,2020-10-30 06:57:19
3,Hilo,19.73,-155.09,77.00,88,90,1.79,US,overcast clouds,2020-10-30 06:57:19
4,Grand Gaube,-20.01,57.66,73.99,52,0,15.01,MU,light rain,2020-10-30 06:57:19
...,...,...,...,...,...,...,...,...,...,...
688,Padang,-0.95,100.35,85.17,65,94,4.47,ID,overcast clouds,2020-10-30 07:00:18
689,Maceio,-9.67,-35.74,73.40,100,75,6.93,BR,broken clouds,2020-10-30 07:00:07
690,Kingston,18.00,-76.79,80.60,78,20,8.05,JM,thunderstorm,2020-10-30 06:57:34
691,Amahai,-3.33,128.92,84.11,68,80,5.75,ID,light rain,2020-10-30 07:00:19


In [9]:
# Reorder the columns
new_columns = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_columns]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Kholodnyy,RU,62.72,147.95,12.58,94,100,2.82,light snow
1,Chuy,UY,-33.70,-53.46,52.90,82,100,5.93,overcast clouds
2,Pisco,PE,-13.70,-76.22,62.01,87,75,13.87,broken clouds
3,Hilo,US,19.73,-155.09,77.00,88,90,1.79,overcast clouds
4,Grand Gaube,MU,-20.01,57.66,73.99,52,0,15.01,light rain
5,Komsomolskiy,UZ,40.43,71.72,53.60,27,0,4.70,clear sky
6,Yulara,AU,-25.24,130.99,84.20,7,37,12.75,scattered clouds
7,Carnarvon,AU,-24.87,113.63,78.80,65,0,19.46,clear sky
8,Nanortalik,GL,60.14,-45.24,29.77,92,100,9.17,overcast clouds
9,Kodiak,US,57.79,-152.41,28.40,79,1,6.93,clear sky


In [10]:
# Export to csv
city_data_df.to_csv("WeatherPy_Database.csv", index_label="City ID")